In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

In [57]:
excel_loc = "test.xlsx"

def train(target_col):
    # read the frame
    frame_raw = pd.read_excel(excel_loc, sheet_name=0, index_col=0).drop(["Run"], axis=1)
    print(frame_raw)
    # minmax scale
    scaler = MinMaxScaler()
    scaler.fit(frame_raw)
    frame_scaled = frame_raw[0:0].copy()
    frame_scaled[:] = scaler.transform(frame_raw)
    print(frame_scaled)
    
    # split
    x, y = frame_scaled.loc[:, frame_raw.columns != target_col], frame_scaled.loc[:, frame_raw.columns == target_col] 
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    
    # train
    svr = SVR(kernel='linear')
    svr.fit(x_train, y_train)
    
    # return weights
    return svr.coef_

In [58]:
train("peak_r0")

   num_infec_agents  num_uninfec_agents  num_rec_agents  mask_efficacy  \
0                 1                  20               0             95   
1                 2                  20               0             95   
2                 3                  20               0             95   
3                 4                  20               0             95   

   length  num_humans  peak_infection  peak_infection_loc  peak_infection_pct  \
0     232          22              22                  87            1.000000   
1     225          23              23                  68            1.000000   
2     272          24              23                  66            0.958333   
3     292          25              23                  86            0.920000   

    peak_r0  peak_r0_loc  
0  2.166667           34  
1  2.000000            2  
2  1.800000           26  
3  1.875000           63  
   num_infec_agents  num_uninfec_agents  num_rec_agents  mask_efficacy  \
0          0.0

A:\Users\bcwarner\Anaconda3\envs\covid\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array([[-0.1509242 ,  0.        ,  0.        ,  0.        , -0.135156  ,
        -0.1509242 , -0.2263863 ,  0.2263863 ,  0.11790953,  0.02969001]])